<a href="https://colab.research.google.com/github/NTU-Sherlock/Stock_price/blob/master/panel_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Panel Practice
A practice to use [Panel](https://panel.holoviz.org/index.html) and [Plotly](https://plotly.com/python/) to create a simple stock price dashboard app.

In [11]:
import os
import datetime
import pandas as pd
import plotly.graph_objects as go

## Stock Market Dataset
* Downloaded from Kaggle: [Stock Market Dataset](https://www.kaggle.com/jacksoncrow/stock-market-dataset)
* This dataset contain historical daily prices for all tickers currently trading on NASDAQ.
* In this pratice I only use 5 tickers (APPL, AMZN, FB, GOOGL, MSFT) for panel demostration.

In [34]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [35]:
%cd /content/drive/MyDrive/Python_Learning/Signal_Prediction/panel_demo


/content/drive/MyDrive/Python_Learning/Signal_Prediction/panel_demo


In [37]:
%ls

data/     panel_practice.html   stock_price_dashboard.py
figures/  panel_practice.ipynb


In [40]:
files = os.listdir('data')
files

['AAPL.csv', 'AMZN.csv', 'FB.csv', 'GOOGL.csv', 'MSFT.csv']

In [42]:
# Read and Preprocessing Data
AAFMG = []
for f in files:
    data = pd.read_csv('data/' + f)
    data['Symbol'] = f.replace('.csv', '')
    AAFMG.append(data)
AAFMG = pd.concat(AAFMG)
AAFMG['Date'] = pd.to_datetime(AAFMG['Date'])

In [43]:
# Check Data
AAFMG.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400,AAPL
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200,AAPL
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000,AAPL
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400,AAPL
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400,AAPL


## Before using Panel:
We can write a function that shows the price trend, moving average curve with parameters: symbol, value to observe, and window of moving average.

In [44]:
def get_plot(symbol = 'AAPL', value = 'Close', window = 30):
    
    ## filter specific symbol and calculate the moving average
    df = AAFMG.loc[AAFMG['Symbol'] == symbol]
    df = df.sort_values('Date')
    df['MA'] = df[value].rolling(window = window).mean()
    df = df.loc[(df['Date'] >= '2017-04-01') & (df['Date'] <= '2020-04-01')]
    
    ## create plotly object
    fig = go.Figure(layout = go.Layout(plot_bgcolor = '#EEEEEE'))
    fig.add_trace(go.Scatter(
        x = df["Date"], y = df[value],
        name = '%s/%s' %(symbol, value),
        fill = 'tozeroy',
        line_width = 0,
        line_color = 'rgba(72,89,110,1)',
        fillcolor = 'rgba(72,89,110,0.8)',
    ))
    fig.add_trace(go.Scatter(
        x = df["Date"], y = df["MA"],
        name = '%s/%s/MA%d' %(symbol, value, window),
        mode = "lines",
        line_color = '#a57571',
        line_width = 2.5
    ))
    fig.update_layout(
        width = 600, height = 400,
        margin=dict(l = 20, r = 20, t = 20, b = 20),
        legend = dict(
            orientation = "h",
            yanchor = "bottom",
            y = 1.02,
            xanchor = "right",
            x = 1
        )
    )
    
    return fig

In [45]:
get_plot(symbol = 'GOOGL', value = 'Open', window = 60)

It works, but exploring all symbols, values, and different window size by passing arguments to this function is tedious and too manual. User may not have ability to use your python function.

## Start using Panel:

In [47]:
import panel as pn
pn.extension("plotly")

we can easily turn our python function into interactive graph with `pn.interact()`

In [48]:
pn.interact(get_plot, symbol = 'AAPL', value = 'Close', window = 30)

Column
    [0] Column
        [0] TextInput(name='symbol', value='AAPL')
        [1] TextInput(name='value', value='Close')
        [2] IntSlider(end=90, name='window', start=-30, value=30)
    [1] Row
        [0] Plotly(Figure, name='interactive00099')

but it still have some drawbacks:
* the value of symbol and value need to key in by user, user may not have information about which symbol and value are available.
* the range of window size contains negative value which doesn't make any sence obviously!
* the layout of dashboard we cannot control.

we can specify the possible value of arguments and pass it into `pn.interact()`

In [49]:
kw = dict(
    symbol = ['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT'],
    value = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
    window = (7, 365)
)

In [50]:
i = pn.interact(get_plot, **kw)

In [ ]:
i

Column
    [0] Column
        [0] Select(name='symbol', options=['AAPL', 'AMZN', ...], value='AAPL')
        [1] Select(name='value', options=['Open', 'High', ...], value='Close')
        [2] IntSlider(end=365, name='window', start=7, value=30, value_throttled=186)
    [1] Row
        [0] Plotly(Figure, name='interactive00019')

## Components of of Panel
take a look of what `pn.interact` creates:

In [ ]:
i.pprint()

Column
    [0] Column
        [0] Select(name='symbol', options=['AAPL', 'AMZN', ...], value='AAPL')
        [1] Select(name='value', options=['Open', 'High', ...], value='Close')
        [2] IntSlider(end=365, name='window', start=7, value=30, value_throttled=186)
    [1] Row
        [0] Plotly(Figure, name='interactive00019')


`pn.interact` created a `pn.Column` object consisting of a WidgetBox with 3 widgets and a `pn.Row` with one Plotly figure object. So our dashboard have layout like that:
<img src="./figures/components_of_panel.png" alt="drawing" width="600"/>

In [ ]:
i[0][0]

Select(name='symbol', options=['AAPL', 'AMZN', ...], value='AAPL')

In [ ]:
i[1][0]

Plotly(Figure, name='interactive00019')

Panel is compositional, so you can recomposite these components any way you like, or adding other objects:

In [ ]:
text = "# Stock Market Data\nSelect a symbol, value, and the time window for moving average"

p = pn.Row(i[1][0], pn.Column(text, i[0][0], i[0][1], i[0][2]))
p.pprint()

Row
    [0] Plotly(Figure, name='interactive00019')
    [1] Column
        [0] Markdown(str)
        [1] Select(name='symbol', options=['AAPL', 'AMZN', ...], value='AAPL')
        [2] Select(name='value', options=['Open', 'High', ...], value='Close')
        [3] IntSlider(end=365, name='window', start=7, value=30, value_throttled=186)


In [ ]:
p

Row
    [0] Plotly(Figure, name='interactive00019')
    [1] Column
        [0] Markdown(str)
        [1] Select(name='symbol', options=['AAPL', 'AMZN', ...], value='AAPL')
        [2] Select(name='value', options=['Open', 'High', ...], value='Close')
        [3] IntSlider(end=365, name='window', start=7, value=30, value_throttled=186)

The widgets stay linked to their plot even if they are in a different notebook cell:

In [ ]:
i[0][2]

IntSlider(end=365, name='window', start=7, value=30, value_throttled=186)

Panel widgets are reactive, so they will update even if you set the values by hand:

In [ ]:
i[0][2].value = 30

## Using Panel Widgets

In [ ]:
import panel.widgets as pnw

There are many [widgets in Panel](https://panel.holoviz.org/user_guide/Widgets.html), here I use `Select`, `RadioButtonGroup`, `IntSlider`, `DateRangeSlider` to build the stock price dashboard.

In [ ]:
# Create Widget Components
symbol = pnw.Select(name = 'symbol', options = ['AAPL', 'AMZN', 'FB', 'GOOGL', 'MSFT'])
value = pnw.RadioButtonGroup(name = 'value', options = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
window = pnw.IntSlider(name = 'window', start = 7, end = 365, value = 30)
date_range = pnw.DateRangeSlider(name = 'date range',
                                 start = datetime.datetime(1980, 12, 12), end = datetime.datetime(2020, 4, 1),
                                 value = (datetime.datetime(2017, 4, 1), datetime.datetime(2020, 4, 1)))

In [ ]:
# Define Reactive Plot Function
@pn.depends(symbol, value, window, date_range)
def reactive_plot(symbol, value, window, date_range):
    df = AAFMG.loc[AAFMG['Symbol'] == symbol]
    df = df.sort_values('Date')
    df['MA'] = df[value].rolling(window = window).mean()
    df = df.loc[(df['Date'] >= date_range[0]) & (df['Date'] <= date_range[1])]
    
    fig = go.Figure(layout = go.Layout(plot_bgcolor = '#EEEEEE'))
    fig.add_trace(go.Scatter(
        x = df["Date"], y = df[value],
        name = '%s/%s' %(symbol, value),
        fill = 'tozeroy',
        line_width = 0,
        line_color = 'rgba(72,89,110,1)',
        fillcolor = 'rgba(72,89,110,0.8)',
    ))
    fig.add_trace(go.Scatter(
        x = df["Date"], y = df["MA"],
        name = '%s/%s/MA%d' %(symbol, value, window),
        mode = "lines",
        line_color = '#a57571',
        line_width = 2.5
    ))
    fig.update_layout(
        width = 600, height = 400,
        margin=dict(l = 20, r = 20, t = 20, b = 20),
        legend = dict(
            orientation = "h",
            yanchor = "bottom",
            y = 1.02,
            xanchor = "right",
            x = 1
        )
    )
    
    return fig

In [ ]:
widgets = pn.Column(text, symbol, value, window, date_range)
dashboard = pn.Row(reactive_plot, widgets)

In [ ]:
dashboard

Row
    [0] ParamFunction(function)
    [1] Column
        [0] Markdown(str)
        [1] Select(name='symbol', options=['AAPL', 'AMZN', ...], value='AAPL')
        [2] RadioButtonGroup(name='value', options=['Open', 'High', ...], value='Open')
        [3] IntSlider(end=365, name='window', start=7, value=30, value_throttled=30)
        [4] DateRangeSlider(end=datetime.datetime(2020, ..., name='date range', start=datetime.datetime(1980, ..., value=(datetime.datetime(2017, ..., value_throttled=(datetime.datetime(2017, ...)

## Deploying Panels
you can simply deploy panel in the separate web page with .servable(), and then run the shell command `panel serve --show panel_practice.ipynb` to launch a server containing that object.

In [ ]:
dashboard.servable();

## Other widgets in Panel

In [ ]:
widgets = pn.Column(
    "### __Categorical - Single Values__",
    pn.Row("* __AutoCompleteInput__: ", pnw.AutocompleteInput(options=['Biology', 'Chemistry', 'Physics'], placeholder='Write something here')),
    pn.Row("* __RadioButtomGroup__: ", pnw.RadioButtonGroup(options=['Biology', 'Chemistry', 'Physics'])),
    pn.Row("* __RadioBoxGroup__: ", pnw.RadioBoxGroup(options=['Biology', 'Chemistry', 'Physics'], inline=True)),
    pn.Row("* __Select__: ", pnw.Select(options=['Biology', 'Chemistry', 'Physics'])),
    
    "### __Categorical - Multiple Values__",
    pn.Row("* __CheckBoxGroup__: ", pnw.CheckBoxGroup(value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'],inline=True)),
    pn.Row("* __CheckButtonGroup__: ", pnw.CheckButtonGroup(value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'])),
    pn.Row("* __CrossSelecter__: ", pnw.CrossSelector(value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'])),
    pn.Row("* __MultiSelect__: ", pnw.MultiSelect(value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'])),
    
    
    "### __Numeric - Int__",
    pn.Row("* __IntSlider__: ", pnw.IntSlider(start=0, end=8, step=2, value=4)),
    pn.Row("* __IntRangeSlider__: ", pnw.IntRangeSlider(start=0, end=10, value=(2, 8), step=2)),
    pn.Row("* __Player__: ", pnw.Player(start=0, end=100, value=32, loop_policy='loop')),
    pn.Row("* __DiscreteSlider__: ", pnw.DiscreteSlider(options=[2, 4, 8, 16, 32, 64, 128], value=32)),
    pn.Row("* __DiscretePlayer__: ", pnw.DiscretePlayer(options=[2, 4, 8, 16, 32, 64, 128], value=32, loop_policy='loop')),
    
    "### __Numeric - Float__",
    pn.Row("* __FloatSlider__: ", pnw.FloatSlider(start=0, end=3.141, step=0.01, value=1.57)),
    pn.Row("* __RangeSlider__: ", pnw.RangeSlider(format='0.0a', start=100000, end=1000000)),
    
    "### __Boolean__",
    pn.Row("* __Checkbox__: ", pnw.Checkbox(name='Checkbox')),
    pn.Row("* __Toogle__: ", pnw.Toggle(name='Toggle')),
    
    "### __Dates__",
    pn.Row("* __DatetimeInput__: ", pnw.DatetimeInput(value=datetime.datetime(2019, 2, 8))),
    pn.Row("* __DatePicker__: ", pnw.DatePicker()),
    pn.Row("* __DateRangeSlider__: ", pnw.DateRangeSlider(start=datetime.datetime(2019, 1, 1), end=datetime.datetime(2019, 6, 1))),
    
    "### __Text__",
    pn.Row("* __TextInput__: ", pnw.TextInput(placeholder='Enter a string here...')),
    pn.Row("* __PasswordInput__: ", pnw.PasswordInput(placeholder='Enter a string here...')),
    pn.Row("* __TextAreaInput__: ", pnw.TextAreaInput(placeholder='Enter a string here...')),
    
    "### __Other__",
    pn.Row("* __ColorPicker__: ", pnw.ColorPicker(value='#99ef78')),
    pn.Row("* __FileInput__: ", pnw.FileInput(accept='.csv,.json')),
    pn.Row("* __LiteralInput__: ", pnw.LiteralInput(value={'key': [1, 2, 3]}, type=dict)),
    pn.Row("* __Button__: ", pnw.Button(name='Click me', button_type='primary')),
    pn.Row("* __DataFrame__: ", pnw.DataFrame(pd.DataFrame({'int': [1, 2, 3], 'float': [3.14, 6.28, 9.42], 'str': ['A', 'B', 'C']}, index=[1, 2, 3]))),
    pn.Row("* __FileDownload__: ", pnw.FileDownload(file='FileDownload.ipynb', filename='custom_filename.ipynb')),
    pn.Row("* __Progress__: ", pnw.Progress(width=200))
)

In [ ]:
widgets

Column
    [0] Markdown(str)
    [1] Row
        [0] Markdown(str)
        [1] AutocompleteInput(options=['Biology', 'Chemistry', ...], placeholder='Write something here')
    [2] Row
        [0] Markdown(str)
        [1] RadioButtonGroup(options=['Biology', 'Chemistry', ...], value='Biology')
    [3] Row
        [0] Markdown(str)
        [1] RadioBoxGroup(inline=True, options=['Biology', 'Chemistry', ...], value='Biology')
    [4] Row
        [0] Markdown(str)
        [1] Select(options=['Biology', 'Chemistry', ...], value='Biology')
    [5] Markdown(str)
    [6] Row
        [0] Markdown(str)
        [1] CheckBoxGroup(inline=True, options=['Apple', 'Banana', ...], value=['Apple', 'Pear'])
    [7] Row
        [0] Markdown(str)
        [1] CheckButtonGroup(options=['Apple', 'Banana', ...], value=['Apple', 'Pear'])
    [8] Row
        [0] Markdown(str)
        [1] CrossSelector(options=['Apple', 'Banana', ...], value=['Apple', 'Pear'])
    [9] Row
        [0] Markdown(str)
        [1] MultiSelect(options=['Apple', 'Banana', ...], value=['Apple', 'Pear'])
    [10] Markdown(str)
    [11] Row
        [0] Markdown(str)
        [1] IntSlider(end=8, step=2, value=4, value_throttled=4)
    [12] Row
        [0] Markdown(str)
        [1] IntRangeSlider(end=10, step=2, value=(2, 8), value_throttled=(2, 8))
    [13] Row
        [0] Markdown(str)
        [1] Player(end=100, loop_policy='loop', value=32)
    [14] Row
        [0] Markdown(str)
        [1] DiscreteSlider(formatter='%d', options=[2, 4, 8, 16, ...], value=32, value_throttled=32)
    [15] Row
        [0] Markdown(str)
        [1] DiscretePlayer(loop_policy='loop', options=[2, 4, 8, 16, ...], value=32)
    [16] Markdown(str)
    [17] Row
        [0] Markdown(str)
        [1] FloatSlider(end=3.141, step=0.01, value=1.57, value_throttled=1.57)
    [18] Row
        [0] Markdown(str)
        [1] RangeSlider(end=1000000, format='0.0a', start=100000, step=1, value=(100000, 1000000), value_throttled=(100000, 1000000))
    [19] Markdown(str)
    [20] Row
        [0] Markdown(str)
        [1] Checkbox()
    [21] Row
        [0] Markdown(str)
        [1] Toggle()
    [22] Markdown(str)
    [23] Row
        [0] Markdown(str)
        [1] DatetimeInput(type=<class 'datetime.datetime'..., value=datetime.datetime(2019, ...)
    [24] Row
        [0] Markdown(str)
        [1] DatePicker()
    [25] Row
        [0] Markdown(str)
        [1] DateRangeSlider(end=datetime.datetime(2019, ..., start=datetime.datetime(2019, ..., value=(datetime.datetime(2019, ..., value_throttled=(datetime.datetime(2019, ...)
    [26] Markdown(str)
    [27] Row
        [0] Markdown(str)
        [1] TextInput(placeholder='Enter a string here...')
    [28] Row
        [0] Markdown(str)
        [1] PasswordInput(placeholder='Enter a string here...')
    [29] Row
        [0] Markdown(str)
        [1] TextAreaInput(placeholder='Enter a string here...')
    [30] Markdown(str)
    [31] Row
        [0] Markdown(str)
        [1] ColorPicker(value='#99ef78')
    [32] Row
        [0] Markdown(str)
        [1] FileInput(accept='.csv,.json')
    [33] Row
        [0] Markdown(str)
        [1] LiteralInput(type=<class 'dict'>, value={'key': [1, 2, 3]})
    [34] Row
        [0] Markdown(str)
        [1] Button(button_type='primary', name='Click me')
    [35] Row
        [0] Markdown(str)
        [1] DataFrame(value=   int  float s...)
    [36] Row
        [0] Markdown(str)
        [1] FileDownload(file='FileDownload.ipynb', filename='custom_filename.ipynb', label='Download custom_filename....)
    [37] Row
        [0] Markdown(str)
        [1] Progress(width=200)